In [1]:
#   %matplotlib inline
'''
在使用 Jupyter Notebook 时，通常在代码的开头加入 %matplotlib inline，以确保在 notebook 中正确显示 Matplotlib 图形。
'''
#导包
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='SimHei', size=13)


import os,gc,re,warnings,sys
warnings.filterwarnings("ignore")

#读取数据
# path = './data/' # 自定义的路径
path = 'D:/Programer/pycharm/MyCode/NewsRec/dataSet/' # 天池平台路径

#####train
trn_click = pd.read_csv(path+'train_click_log.csv')
# 指定哪些列是我想要读取的
# trn_click = pd.read_csv(path+'train_click_log.csv', names=['user_id','item_id','click_time','click_environment','click_deviceGroup','click_os','click_country','click_region','click_referrer_type'])
item_df = pd.read_csv(path+'articles.csv')
print("item_df")
print(item_df)
print("--------------------------------")
item_df = item_df.rename(columns={'article_id': 'click_article_id'})  #重命名，方便后续match
item_emb_df = pd.read_csv(path+'articles_emb.csv')
print("item_emb_df")
print(item_emb_df)
print("--------------------------------")
#####test
tst_click = pd.read_csv(path+'testA_click_log.csv')
print("tst_click")
print(tst_click)
print("--------------------------------")

# 数据预处理
# 计算用户点击rank和点击次数
    # 对每个用户的点击时间戳进行排序
trn_click['rank'] = trn_click.groupby(['user_id'])['click_timestamp'].rank(ascending=False).astype(int)
tst_click['rank'] = tst_click.groupby(['user_id'])['click_timestamp'].rank(ascending=False).astype(int)

    #计算用户点击文章的次数，并添加新的一列count
trn_click['click_cnts'] = trn_click.groupby(['user_id'])['click_timestamp'].transform('count')
tst_click['click_cnts'] = tst_click.groupby(['user_id'])['click_timestamp'].transform('count')

# 数据浏览
print("trn_click 数据浏览:")
df_trn_click = pd.DataFrame(trn_click)
print(df_trn_click.head(20))
print("================================")
print(df_trn_click.columns)
print(trn_click)
print("trn_click")
df_trn_click = pd.DataFrame(trn_click)
print(df_trn_click.head(20))
print("================================")
print(df_trn_click.columns)
print(trn_click)
print("--------------------------------")
print("tst_click")
print(tst_click)
print("--------------------------------")
print("item_df:")
print(item_df)
print("--------------------------------")

    #用户点击日志文件_训练集
'''
@:param 左边为trn_click,右边为item_df，，，how表示连接方式，如果是left，则左边的全保留，右边的进行全映射的配对，相同的on，则右边的其他属性和左边的属性合并merge，成一个大，新的trn_click
'''
trn_click = trn_click.merge(item_df, how='left', on=['click_article_id'])
print(trn_click.head(20))
print("================================")
print(trn_click.columns)
print("--------------------------------")

'''
train_click_log.csv文件数据中每个字段的含义:
    user_id: 用户的唯一标识
    click_article_id: 用户点击的文章唯一标识
    click_timestamp: 用户点击文章时的时间戳
    click_environment: 用户点击文章的环境
    click_deviceGroup: 用户点击文章的设备组
    click_os: 用户点击文章时的操作系统
    click_country: 用户点击文章时的所在的国家
    click_region: 用户点击文章时所在的区域
    click_referrer_type: 用户点击文章时，文章的来源
'''

#用户点击日志信息
print("-----------------------------------------------------------------------------------------------------------------")
print("trn_click.info():")
trn_click.info()

print("-----------------------------------------------------------------------------------------------------------------")
print("trn_click.describe():")
print(trn_click.describe())


#训练集中的用户数量为20w
print("训练用户数量：",end="")
print(trn_click.user_id.nunique())

print("用户最少点击文本数量：",end="")
print(trn_click.groupby('user_id')['click_article_id'].count().min() )# 训练集里面每个用户至少点击了两篇文章


# plt.figure()
# plt.figure(figsize=(15, 20))
# i = 1
# for col in ['click_article_id', 'click_timestamp', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country','click_region', 'click_referrer_type', 'rank', 'click_cnts']:
#     plot_envs = plt.subplot(5, 2, i)
#     i += 1
#     v = trn_click[col].value_counts().reset_index()[:10]
#     print(v)
#     print("----------------------------------")
#     fig = sns.barplot(data=staff,x=v['index'], y=v[col],hue=)   # fig = sns.barplot(x=v['index'], y=v[col])
#     for item in fig.get_xticklabels():
#         item.set_rotation(90)
#     plt.title(col)
# plt.tight_layout()
#plt.show()
  ### 换种显示方式还是不行啊，难搞 = =~
plt.figure()
# plt.figure(figsize=(15, 20))
# i = 1
# for col in ['click_article_id', 'click_timestamp', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type', 'rank', 'click_cnts']:
#     plot_envs = plt.subplot(5, 2, i)
#     i += 1
#     v = trn_click[col].value_counts().reset_index()[:10]
#     v = pd.DataFrame(v)
#     print(v)
#     print("----------------------------------")
#     fig = plt.bar(v['index'], v[col])
#     plt.xticks(rotation=90)
#     plt.title(col)
#
# plt.tight_layout()
# plt.show()

#
#
print("以用户角度分析，画出每个用户点击文章次数的直方图:")
print(trn_click['click_environment'].value_counts())

print("从点击设备组click_deviceGroup来看，设备1占大部分（61%），设备3占36%。")
print(trn_click['click_deviceGroup'].value_counts())

print("测试集用户点击日志:")
tst_click = tst_click.merge(item_df, how='left', on=['click_article_id'])
print(tst_click.head())

print("tsr_click.describe()描述:")
print(tst_click.describe())
#
#测试集中的用户数量为5w
print(tst_click.user_id.nunique()) #50000
print("测试集用户最少点击文本数量：",end="")
tst_click.groupby('user_id')['click_article_id'].count().min() # 注意测试集里面有只点击过一次文章的用户
print(tst_click.groupby('user_id')['click_article_id'].count().min() )# 训练集里面每个用户至少点击了1篇文章

# # 新闻文章信息数据表
item_df.head()._append(item_df.tail())
print("新闻文章信息数据集浏览:")
print(item_df.head(20))

# 对单词数进行记录统计
item_df['words_count'].value_counts()
print(item_df['words_count'].value_counts())

# 查看文章有多少种类型
print(item_df["category_id"].nunique())   #461
# item_df["category_id"].hist()
# 显示直方图
# x = item_df['category_id'].nunique()
# y = item_df["category_id"].hist
# plt.plot(x, y)
# plt.show()

# 用户重复点击
#####merge
user_click_merge = trn_click._append(tst_click)
print(user_click_merge.head())

#用户重复点击
user_click_count = user_click_merge.groupby(['user_id', 'click_article_id'])['click_timestamp'].agg({'count'}).reset_index()
print(user_click_count[:10])    #x或者叫“行” 显示哪些 ？    y或者叫“列”显示哪些 ？


print(user_click_count[user_click_count['count']>7])

print(user_click_count['count'].unique())

print(user_click_count.loc[:,'count'].value_counts())

# 用户点击环境变化分析
# def plot_envs(df, cols, r, c):
#     plt.figure()
#     plt.figure(figsize=(10, 5))
#     i = 1
#     for col in cols:
#         plt.subplot(r, c, i)
#         i += 1
#         v = df[col].value_counts().reset_index()
#         fig = sns.barplot(x=v['index'], y=v[col])
#         for item in fig.get_xticklabels():
#             item.set_rotation(90)
#         plt.title(col)
#     plt.tight_layout()
#     plt.show()
'''
    raise KeyError(key) from err
        KeyError: 'index'
    sns.barplot(x=v['index'], y=v[col]) 画图报错，暂时不清楚怎么解决 = =~
'''

# # 分析用户点击环境变化是否明显，这里随机采样10个用户分析这些用户的点击环境分布
# sample_user_ids = np.random.choice(tst_click['user_id'].unique(), size=10, replace=False)
# sample_users = user_click_merge[user_click_merge['user_id'].isin(sample_user_ids)]
# cols = ['click_environment','click_deviceGroup', 'click_os', 'click_country', 'click_region','click_referrer_type']
# for _, user_df in sample_users.groupby('user_id'):
#     plot_envs(user_df, cols, 2, 3)

# 用户点击新闻数量的分布
user_click_item_count = sorted(user_click_merge.groupby('user_id')['click_article_id'].count(), reverse=True)
plt.plot(user_click_item_count)
plt.show()








item_df
        article_id  category_id  created_at_ts  words_count
0                0            0  1513144419000          168
1                1            1  1405341936000          189
2                2            1  1408667706000          250
3                3            1  1408468313000          230
4                4            1  1407071171000          162
...            ...          ...            ...          ...
364042      364042          460  1434034118000          144
364043      364043          460  1434148472000          463
364044      364044          460  1457974279000          177
364045      364045          460  1515964737000          126
364046      364046          460  1505811330000          479

[364047 rows x 4 columns]
--------------------------------
item_emb_df
        article_id     emb_0     emb_1     emb_2     emb_3     emb_4  \
0                0 -0.161183 -0.957233 -0.137944  0.050855  0.830055   
1                1 -0.523216 -0.974058  0.738608  0.1552

            user_id  click_article_id  click_timestamp  click_environment  \
count  1.087229e+06      1.087229e+06     1.087229e+06       1.087229e+06   
mean   1.219934e+05      1.954501e+05     1.507592e+12       3.947997e+00   
std    5.589688e+04      9.303222e+04     1.484853e+09       3.271162e-01   
min    0.000000e+00      1.000000e+00     1.300000e+01       1.000000e+00   
25%    7.804800e+04      1.239090e+05     1.507293e+12       4.000000e+00   
50%    1.317790e+05      2.058660e+05     1.507614e+12       4.000000e+00   
75%    1.707550e+05      2.774920e+05     1.507848e+12       4.000000e+00   
max    1.999990e+05      3.640460e+05     1.510603e+12       4.000000e+00   

       click_deviceGroup      click_os  click_country  click_region  \
count       1.087228e+06  1.087228e+06   1.087228e+06  1.087228e+06   
mean        1.805232e+00  1.307099e+01   1.309130e+00  1.813411e+01   
std         1.032809e+00  6.935679e+00   1.614191e+00  7.106333e+00   
min         1.000000e+

   user_id  click_article_id  count
0        0             30760      1
1        0            157507      1
2        1             63746      1
3        1            289197      1
4        2             36162      1
5        2            168401      1
6        3             36162      1
7        3             50644      1
8        4             39894      1
9        4             42567      1
        user_id  click_article_id  count
311243    86295             74254     10
311244    86295             76268     10
393762   103237            205948     10
393764   103237            235689     10
558762   134850             69463     13
[ 1  2  4  3  6  5 10  7 13]
count
1     1556606
2       10846
3         390
4          66
5          24
6           8
10          4
7           3
13          1
Name: count, dtype: int64


In [2]:
print(item_df['category_id'].nunique())     # 461个文章主题
item_df['category_id'].hist()

461


<Axes: >

In [3]:
print(item_df["category_id"].nunique())   #461
item_df["category_id"].hist(color='skyblue')
plt.show()

461


In [5]:
#点击次数在前50的用户
plt.plot(user_click_item_count[:50])
plt.show()

In [4]:
#点击次数排名在[25000:50000]之间
plt.plot(user_click_item_count[25000:50000])
plt.show()

In [6]:
# 新闻点击次数分析
item_click_count = sorted(user_click_merge.groupby('click_article_id')['user_id'].count(), reverse=True)
plt.plot(item_click_count)
plt.show()

In [7]:
plt.plot(item_click_count[:100])
plt.show()

In [8]:
plt.plot(item_click_count[:20])
plt.show()

In [9]:
plt.plot(item_click_count[3500:])
plt.show()

## 新闻共现频次：两篇新闻连续出现的次数

In [2]:
import time

In [3]:
start_time = time.time()
tmp = user_click_merge.sort_values('click_timestamp')
tmp['next_item'] = tmp.groupby(['user_id'])['click_article_id'].transform(lambda x:x.shift(-1))
print(tmp)
# 记录程序结束时间
end_time = time.time()

# 计算程序执行时间
execution_time = end_time - start_time

print(f"程序执行时间为: {execution_time} 秒")  #花费
# union_item = tmp.groupby(['click_article_id','next_item'])['click_timestamp'].agg({'count'}).reset_index().sort_values('count', ascending=False)
# union_item[['count']].describe()

NameError: name 'user_click_merge' is not defined

由统计数据可以看出，平均共现次数3.18，最高为2202。

说明用户看的新闻，相关性是比较强的。

In [ ]:
#画个图直观地看一看
x = union_item['click_article_id']
y = union_item['count']
plt.scatter(x, y)
plt.show()

In [32]:
plt.plot(union_item['count'].values[40000:])
plt.show()

大概有75000个pair至少共现一次

## 新闻文章信息

In [ ]:
#不同类型的新闻出现的次数
plt.plot(user_click_merge['category_id'].value_counts().values)
plt.show()

In [ ]:
#出现次数比较少的新闻类型, 有些新闻类型，基本上就出现过几次
plt.plot(user_click_merge['category_id'].value_counts().values[150:])
plt.show()

In [35]:
#新闻字数的描述性统计
user_click_merge['words_count'].describe()

count    1.579974e+06
mean     2.041537e+02
std      6.418569e+01
min      0.000000e+00
25%      1.720000e+02
50%      1.970000e+02
75%      2.300000e+02
max      6.690000e+03
Name: words_count, dtype: float64

In [ ]:
plt.plot(user_click_merge['words_count'].values)
plt.show()

## 用户点击的新闻类型的偏好
此特征可以用于度量用户的兴趣是否广泛。

In [ ]:
plt.plot(sorted(user_click_merge.groupby('user_id')['category_id'].nunique(), reverse=True))
plt.show()

In [38]:
user_click_merge.groupby('user_id')['category_id'].nunique().reset_index().describe()

,user_id,category_id
count,246958.000000,246958.000000
mean,125180.653451,4.526749
std,72593.231202,4.349710
min,0.000000,1.000000
25%,61739.250000,2.000000
50%,126520.500000,3.000000
75%,188259.750000,6.000000
max,249999.000000,95.000000


### 用户查看文章的长度的分布
通过统计不同用户点击新闻的平均字数，这个可以反映用户是对长文更感兴趣还是对短文更感兴趣。

In [ ]:
plt.plot(sorted(user_click_merge.groupby('user_id')['words_count'].mean(), reverse=True))
plt.show()

In [ ]:
#挑出大多数人的区间仔细看看
plt.plot(sorted(user_click_merge.groupby('user_id')['words_count'].mean(), reverse=True)[1000:45000])
plt.show()

In [41]:
#更加详细的参数
user_click_merge.groupby('user_id')['words_count'].mean().reset_index().describe()

,user_id,words_count
count,246958.000000,246958.000000
mean,125180.653451,205.841856
std,72593.231202,47.653032
min,0.000000,8.000000
25%,61739.250000,187.000000
50%,126520.500000,201.937500
75%,188259.750000,218.000000
max,249999.000000,3434.500000


## 用户点击新闻的时间分析

In [ ]:
#为了更好的可视化，这里把时间进行归一化操作
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
user_click_merge['click_timestamp'] = mm.fit_transform(user_click_merge[['click_timestamp']])
user_click_merge['created_at_ts'] = mm.fit_transform(user_click_merge[['created_at_ts']])

user_click_merge = user_click_merge.sort_values('click_timestamp')

In [ ]:
user_click_merge.head()

In [ ]:
def mean_diff_time_func(df, col):
    df = pd.DataFrame(df, columns={col})
    df['time_shift1'] = df[col].shift(1).fillna(0)
    df['diff_time'] = abs(df[col] - df['time_shift1'])
    return df['diff_time'].mean()

In [ ]:
# 点击时间差的平均值
mean_diff_click_time = user_click_merge.groupby('user_id')['click_timestamp', 'created_at_ts'].apply(lambda x: mean_diff_time_func(x, 'click_timestamp'))

In [ ]:
plt.plot(sorted(mean_diff_click_time.values, reverse=True))
plt.show()

In [ ]:
# 前后点击文章的创建时间差的平均值
mean_diff_created_time = user_click_merge.groupby('user_id')['click_timestamp', 'created_at_ts'].apply(lambda x: mean_diff_time_func(x, 'created_at_ts'))

In [ ]:
plt.plot(sorted(mean_diff_created_time.values, reverse=True))
plt.show()

In [ ]:
from gensim.models import Word2Vec
import logging, pickle

# 需要注意这里模型只迭代了一次
def trian_item_word2vec(click_df, embed_size=16, save_name='item_w2v_emb.pkl', split_char=' '):
    click_df = click_df.sort_values('click_timestamp')
    # 只有转换成字符串才可以进行训练
    click_df['click_article_id'] = click_df['click_article_id'].astype(str)
    # 转换成句子的形式
    docs = click_df.groupby(['user_id'])['click_article_id'].apply(lambda x: list(x)).reset_index()
    docs = docs['click_article_id'].values.tolist()

    # 为了方便查看训练的进度，这里设定一个log信息
    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)

    # 这里的参数对训练得到的向量影响也很大,默认负采样为5
    w2v = Word2Vec(docs, size=16, sg=1, window=5, seed=2020, workers=24, min_count=1, iter=10)
    
    # 保存成字典的形式
    item_w2v_emb_dict = {k: w2v[k] for k in click_df['click_article_id']}
    
    return item_w2v_emb_dict

In [ ]:
item_w2v_emb_dict = trian_item_word2vec(user_click_merge)

In [ ]:
# 随机选择5个用户，查看这些用户前后查看文章的相似性
sub_user_ids = np.random.choice(user_click_merge.user_id.unique(), size=15, replace=False)
sub_user_info = user_click_merge[user_click_merge['user_id'].isin(sub_user_ids)]

sub_user_info.head()

In [ ]:
# 上一个版本，这个函数使用的是赛题提供的词向量，但是由于给出的embedding并不是所有的数据的embedding，所以运行下面画图函数的时候会报keyerror的错误
# 为了防止出现这个错误，这里修改为使用word2vec训练得到的词向量进行可视化
def get_item_sim_list(df):
    sim_list = []
    item_list = df['click_article_id'].values
    for i in range(0, len(item_list)-1):
        emb1 = item_w2v_emb_dict[str(item_list[i])] # 需要注意的是word2vec训练时候使用的是str类型的数据
        emb2 = item_w2v_emb_dict[str(item_list[i+1])]
        sim_list.append(np.dot(emb1,emb2)/(np.linalg.norm(emb1)*(np.linalg.norm(emb2))))
    sim_list.append(0)
    return sim_list

In [ ]:
for _, user_df in sub_user_info.groupby('user_id'):
    item_sim_list = get_item_sim_list(user_df)
    plt.plot(item_sim_list)